# Example for:
Transfer Learning Empirical Experiment from Classes: [5, 6, 7, 8, 9](pretraining) to classes: [0,,1, 2, 3, 4] (finetuning) of CIFAR (Random experiment)

### Setup and Hyperparams

In [1]:
# Specify which gpu
import os
gpu_id = 1
os.environ["CUDA_VISIBLE_DEVICES"] = str(gpu_id)

import sys
sys.path.append('/mnt/c/Users/Arnisa/Desktop/MP/mp-tl-study')
from functions.utils import *

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

np.random.seed(0)
torch.manual_seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
if torch.cuda.is_available():
    torch.cuda.manual_seed(0)
    torch.cuda.manual_seed_all(0)  # if using multi-GPU

In [3]:
cuts = [0,1,2,3,4,5,6]
params = {
      # MODEL ARCHITECTURE PARAMS
      'depth': 6,
      'num_channels': 64,
      'activation_function': nn.ReLU,
      'kernel_size': 3,
      # TRAINING PARAMS
      'device': device,
      'lr_pretrain': 0.001,   
      'lr_fine_tune': 0.001, 
      'num_train': 40,
      'early_stop_patience': 6,
      'save_best': False,
      'save_checkpoints': False,
      'is_cnn': True,
      'is_debug': False,
      'classification_report_flag': False,
      'batch_size':64,
      # DATASET PARAMS
      'pre_train_classes': [0, 1, 8, 9],
      'fine_tune_classes': [2, 3, 4, 5, 6, 7],
      'val_split': 0.1,
      'num_workers': 0,
      'generate_dataset_seed': 42,
      # EXPERIMENT SETTING PARAMS
      'use_pooling': True,  
      'pooling_every_n_layers': 2, # add pooling after every n layers specified here. For only one pooling after all the CNN layers, this equals params['depth']
      'pooling_stride': 2,
      'freeze': True,         # VARIABLE
      'reinit': True,         # VARIABLE
      'truncate': False,      # VARIABLE
    }

In [4]:
root_dir = './data' 
transform=transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
            ])
dataloader_wrapped = TransferLearningWrapper(params, datasets.CIFAR10, datasets.CIFAR10, root_dir, transform=transform)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


## Pretraining

In [5]:
pretrained_model = CustomCNN(params, dataloader_wrapped.output_dim, tuple(dataloader_wrapped.finetune_test_loader.dataset[0][0].shape))
pretrained_model.to(device)

CustomCNN(
  (conv0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act0): ReLU()
  (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act1): ReLU()
  (pool1): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act2): ReLU()
  (conv3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act3): ReLU()
  (pool3): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (conv4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act4): ReLU()
  (conv5): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act5): ReLU()
  (pool5): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (fc): Linear(in_features=1024, out_features=5, bias=True)
)

In [13]:
# Train and evaluate
trainer = Trainer(pretrained_model, dataloader_wrapped, params["lr_pretrain"], params)
train_acc, test_acc, effective_epochs, checkpoints = trainer.train()

print(f"Final Training Accuracy: {train_acc:.4f}")
print(f"Final Test Accuracy: {test_acc:.4f}")

Epoch: 0 	Training Accuracy: 71.01%
Validation Accuracy: 71.40%
Epoch: 1 	Training Accuracy: 78.06%
Validation Accuracy: 77.40%
Epoch: 2 	Training Accuracy: 81.28%
Validation Accuracy: 79.92%
Epoch: 3 	Training Accuracy: 85.44%
Validation Accuracy: 83.28%
Epoch: 4 	Training Accuracy: 87.00%
Validation Accuracy: 85.72%
Epoch: 5 	Training Accuracy: 89.88%
Validation Accuracy: 87.68%
Epoch: 6 	Training Accuracy: 89.77%
Validation Accuracy: 86.92%
Epoch: 7 	Training Accuracy: 91.00%
Validation Accuracy: 86.92%
Epoch: 8 	Training Accuracy: 93.44%
Validation Accuracy: 88.84%
Epoch: 9 	Training Accuracy: 92.48%
Validation Accuracy: 87.92%
Final Training Accuracy: 0.9248
Final Test Accuracy: 0.8816


In [14]:
# save model
torch.save(pretrained_model.state_dict(), 'modelcifar10_mixed.pth')

In [6]:
pretrained_model.load_state_dict(torch.load('pretrained_models/random_classes.pth'))
pretrained_model.to(device)

CustomCNN(
  (conv0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act0): ReLU()
  (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act1): ReLU()
  (pool1): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act2): ReLU()
  (conv3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act3): ReLU()
  (pool3): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (conv4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act4): ReLU()
  (conv5): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act5): ReLU()
  (pool5): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (fc): Linear(in_features=1024, out_features=5, bias=True)
)

In [7]:
eval(pretrained_model, device, dataloader_wrapped.test_loader, debug=True, classification_report_flag=True, is_cnn=True)


Average loss: 0.3366, Accuracy: 2198.0/2500 (88%)

              precision    recall  f1-score   support

     Class 0       0.79      0.86      0.83       489
     Class 1       0.97      0.82      0.89       500
     Class 2       0.85      0.86      0.85       509
     Class 3       0.87      0.97      0.92       522
     Class 4       0.94      0.89      0.91       480

    accuracy                           0.88      2500
   macro avg       0.88      0.88      0.88      2500
weighted avg       0.88      0.88      0.88      2500



0.8792

In [14]:
# save model for later use
torch.save(pretrained_model.state_dict(), 'pretrained_models/random_classes.pth')

## Fine-tuning Experiments

### Baselines
We also reuse the baselines a lot! so skip if we already have the jsons

In [9]:
baselines_results = []
percentages = [0.001, 0.01, 0.1, 0.5, 1.0]

In [20]:
dataloader_wrapped.update_phase('finetune')

for sampled_percentage in percentages:      
    if sampled_percentage <= 0.01:
        repeats = 25
    elif sampled_percentage < 0.5:
        repeats = 20
    else:
        repeats = 5
    
    for repeat in range(repeats):
        print(f"\nSampled Percentage: {sampled_percentage}, Lr: {params['lr_fine_tune']}, Repeat: {repeat}")

        # Reduce the dataset
        train_loader_reduced = reduce_dataset(dataloader_wrapped.train_loader, sampled_percentage, seed = repeat)
        torch.manual_seed(repeat)
        dataset_namespace_new = SimpleNamespace(train_loader=train_loader_reduced, test_loader=dataloader_wrapped.test_loader, val_loader=dataloader_wrapped.val_loader)

        # Copy and then cut the model - we already deepcopy it in the function: pretrained_model
        params_tmp = copy.deepcopy(params)
        params_tmp["reinit"] = True
        model_new = cut_custom_cnn_model(pretrained_model, cut_point=0, params=params_tmp, output_dim=dataloader_wrapped.output_dim)
        model_new.to(device)

        # Train and evaluate
        trainer = Trainer(model_new, dataset_namespace_new, params['lr_fine_tune'], params)
        train_acc, test_acc, effective_epochs, checkpoints = trainer.train(verbose=0)
        print(f"Training Accuracy: {train_acc:.4f}, Test Accuracy: {test_acc:.4f}")

        # Store the results
        baselines_results.append({"lr":params['lr_fine_tune'], "sampled_percentage":sampled_percentage, "sampled_cut_point":-1, "repeat":repeat, "train_acc":train_acc, "test_acc":test_acc}) # -1 for the cut point means it's baseline


Sampled Percentage: 0.001, Lr: 0.001, Repeat: 0
Training Accuracy: 0.7500, Test Accuracy: 0.2988

Sampled Percentage: 0.001, Lr: 0.001, Repeat: 1
Training Accuracy: 0.8125, Test Accuracy: 0.3370

Sampled Percentage: 0.001, Lr: 0.001, Repeat: 2
Early stopping invoked.
Training Accuracy: 0.5000, Test Accuracy: 0.2908

Sampled Percentage: 0.001, Lr: 0.001, Repeat: 3
Early stopping invoked.
Training Accuracy: 0.6250, Test Accuracy: 0.3850

Sampled Percentage: 0.001, Lr: 0.001, Repeat: 4
Early stopping invoked.
Training Accuracy: 0.3750, Test Accuracy: 0.2690

Sampled Percentage: 0.001, Lr: 0.001, Repeat: 5
Early stopping invoked.
Training Accuracy: 0.5625, Test Accuracy: 0.3078

Sampled Percentage: 0.001, Lr: 0.001, Repeat: 6
Training Accuracy: 0.8125, Test Accuracy: 0.4080

Sampled Percentage: 0.001, Lr: 0.001, Repeat: 7
Training Accuracy: 0.8750, Test Accuracy: 0.3115

Sampled Percentage: 0.001, Lr: 0.001, Repeat: 8
Early stopping invoked.
Training Accuracy: 0.7500, Test Accuracy: 0.405

In [9]:
# save baseline results
params_tmp = copy.deepcopy(params)
del params_tmp["device"]
params_tmp["activation_function"] = str(params_tmp["activation_function"])
results = [params_tmp] + baselines_results

with open(f'results/baselines_random_classes.json', 'w') as f:
    json.dump(results, f)

### Fine-tuning

In [13]:
results = []
percentages = [0.001, 0.01, 0.1, 0.5, 1.0]

In [15]:
dataloader_wrapped.update_phase('finetune')

for sampled_percentage in percentages:

    if sampled_percentage <= 0.01:
        repeats = 25
    elif sampled_percentage < 0.5:
        repeats = 20
    else:
        repeats = 5
        
    for sampled_cut_point in cuts:

        for repeat in range(repeats):
            print(f"\nSampled Percentage: {sampled_percentage}, Sampled Cut Point: {sampled_cut_point}, Lr: {params['lr_fine_tune']}, Repeat: {repeat}")

            # Reduce the dataset
            train_loader_reduced = reduce_dataset(dataloader_wrapped.train_loader, sampled_percentage, seed=repeat)
            dataset_namespace_new = SimpleNamespace(train_loader=train_loader_reduced, test_loader=dataloader_wrapped.test_loader, val_loader=dataloader_wrapped.val_loader)
            torch.manual_seed(repeat) # because in the cut function we reinitialize some layers too (at least the dense layers)
            
            # Copy and then cut the model - we already deepcopy it in the function: pretrained_model
            model_new = cut_custom_cnn_model(pretrained_model, cut_point=sampled_cut_point, params=params, output_dim=dataloader_wrapped.output_dim)
            model_new.to(device)
            
            # Train and evaluate
            trainer = Trainer(model_new, dataset_namespace_new, params['lr_fine_tune'], params)
            train_acc, test_acc, effective_epochs, checkpoints = trainer.train(verbose=0)
            print(f"Training Accuracy: {train_acc:.4f}, Test Accuracy: {test_acc:.4f}")

            # Store the results
            results.append({"lr":params['lr_fine_tune'], "sampled_percentage":sampled_percentage, "sampled_cut_point":sampled_cut_point, "repeat":repeat, "train_acc":train_acc, "test_acc":test_acc})


Sampled Percentage: 0.005, Sampled Cut Point: 0, Lr: 0.001, Repeat: 0


Training Accuracy: 0.3939, Test Accuracy: 0.2300

Sampled Percentage: 0.005, Sampled Cut Point: 1, Lr: 0.001, Repeat: 0
Early stopping invoked.
Training Accuracy: 0.1667, Test Accuracy: 0.1667

Sampled Percentage: 0.005, Sampled Cut Point: 2, Lr: 0.001, Repeat: 0
Early stopping invoked.
Training Accuracy: 0.3106, Test Accuracy: 0.2325

Sampled Percentage: 0.005, Sampled Cut Point: 3, Lr: 0.001, Repeat: 0
Training Accuracy: 0.5076, Test Accuracy: 0.2987

Sampled Percentage: 0.005, Sampled Cut Point: 4, Lr: 0.001, Repeat: 0
Training Accuracy: 0.5152, Test Accuracy: 0.3090

Sampled Percentage: 0.005, Sampled Cut Point: 5, Lr: 0.001, Repeat: 0
Training Accuracy: 0.7424, Test Accuracy: 0.3370

Sampled Percentage: 0.005, Sampled Cut Point: 6, Lr: 0.001, Repeat: 0
Training Accuracy: 0.6212, Test Accuracy: 0.2985

Sampled Percentage: 0.01, Sampled Cut Point: 0, Lr: 0.001, Repeat: 0
Training Accuracy: 0.4296, Test Accuracy: 0.2897

Sampled Percentage: 0.01, Sampled Cut Point: 1, Lr: 0.001, Repe

In [30]:
# save fine-tuning results
params_tmp = copy.deepcopy(params)
del params_tmp["device"]
params_tmp["activation_function"] = str(params_tmp["activation_function"])
results = [params_tmp] + results

with open(f'results/cuts_random_classes_FR.json', 'w') as f:
    json.dump(results, f)